In [2]:
# librerías necesarias para realizar este estudio.
#librerías generales
import pandas as pd
import numpy as np
import math

import plotly.express as px
from plotly import colors
import plotly.figure_factory as ff
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import signal
import sys
import os
import numpy as np


# Carga del Dataset
#Variables genéricas:
path1=r'C:\Users\Nitropc\IT Academy\Data Science\Proyecto Data Science\Fuente de Datos\heart.csv'#
#path1=r'/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv'
path2=r'C:\Users\Nitropc\IT Academy\Data Science\Proyecto Data Science\Fuente de Datos\o2Saturation.csv'
#Funciones
# Intento de Mecanizar acciones/herramientas para el analisis de cualquier heartSet
# Guardar información de los pasos EDA


#Cuerpo 
#heart = pd.read_csv(path,sep=',',encoding='ISO-8859-1')
heart = pd.read_csv(path1,sep=',',encoding='latin-1')
heart_columns=['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'target']
heart.columns = heart_columns

heart.columns
o2sat = pd.read_csv(path2,sep=',',encoding='latin-1')
o2sat.columns=['Sat_level']

#Añadimos la columna del dataset o2sat a heart
heart["Sat_level"] = o2sat["Sat_level"]
#Reordenar atributos
heart = heart[['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall','Sat_level', 'target']]
#Definir Colores
color_palette = colors.qualitative.Plotly
cols = px.colors.DEFAULT_PLOTLY_COLORS
colors= color_palette

In [5]:



# Crear un DataFrame de ejemplo
df = pd.DataFrame(heart)


# Definir diccionarios de conversión para los atributos
# Definir diccionarios de conversión para los atributos
lista = ['age','trtbps', 'chol', 'thalachh', 'oldpeak', 'thall', 'Sat_level']

categoricos = ['sex','cp','fbs','restecg','exng','slp','caa','thall','target']


conversion_dict = {
'age': {
        'title': 'Age',
        'values': {'min': df['age'].min(), 'max': df['age'].max()}},
'sex': {'title': 'Gender',
        'values': {0: 'Female(0)', 1: 'Male(1)'}}, #genero
'exng': {'title': 'Exercise-induced Angina',
         'values': {0:'No(0)', 1:'Yes(1)'}}, #angina inducida por el ejercicio 
'caa': {'title': 'Number of major vessels',
        'values': {0:'vessels(0)', 1:'vessels(1)',2:'vessels(2)', 3:'vessels(3)',4:'vessels(4)'}}, #number of major vessels
'cp': {'title': 'Chest Pain type chest pain type',
       'values': {0:'Typical angina(0)', 1:'Atypical angina(1)', 2:'Non-anginal pain(2)', 3:'Asymptomatic(3)', 4:'(4)'}}, #Chest Pain type chest pain type
'fbs': {'title': 'Fasting blood sugar > 120 mg/dl',
        'values': {0:'<121mg/dl(0)', 1:'>120mg/dl(1)'}}, #fasting blood sugar > 120 mg/dl
'restecg': {'title': 'Resting electrocardiographic results',
            'values': {0:'Normal(0)', 1:'Having ST-T wave abnormality(1)',2:'Hypertrophy(2)'}}, # resting electrocardiographic results
    # Agrega otros diccionarios de conversión para los atributos adicionales
'trtbps': {
        'title': 'Resting blood pressure (in mm Hg)',
        'values': {'min': df['trtbps'].min(), 'max': df['trtbps'].max()}},  
'chol': {
        'title': 'Cholestoral in mg/dl fetched via BMI sensor',
        'values': {'min': df['chol'].min(), 'max': df['chol'].max()}}, 
'thalachh': {
        'title': 'Maximum heart rate achieved',
        'values': {'min': df['thalachh'].min(), 'max': df['thalachh'].max()}}, 
'oldpeak': {
        'title': 'Previous peak',
        'values': {'min': df['oldpeak'].min(), 'max': df['oldpeak'].max()}},
'slp': {'title': 'Slope of the peak exercise',
        'values': {0:'Donwsloping(0)', 1:'Flat(1)', 2:'Upsloping(2)'}},
'thall': {
        'title': 'Thalium Stress Test result',
        'values': {'min': df['thall'].min(), 'max': df['thall'].max()}},
'Sat_level': {
        'title': 'O2 saturation',
        'values': {'min': df['Sat_level'].min(), 'max': df['Sat_level'].max()}},
'target': {'title': 'Heart attack chance',
        'values': {0:'less chance of heart attack(0)', 1:'More chance of heart attack(1)'}}
}

# Crear la aplicación Dash con suppress_callback_exceptions=True
app = dash.Dash(__name__)

# Estilos CSS para los componentes HTML
attribute_name_style = {'color': 'blue'}
attribute_title_style = {'color': 'black'}
attribute_values_style = {'color': 'red'}

# Diseño de la aplicación
app.layout = html.Div([
    html.H1("Estudio de Atributos"),
    html.Div(id='attribute-values'),
    html.H2('Tabla de Valores'),
    html.Div(id='attribute-table'),
    html.Div([
        dcc.Dropdown(
            id='x-axis',
            options=[{'label': col, 'value': col} for col in df.columns],
            value=df.columns[0]
        )
    ],
        style={'width': '48%', 'display': 'inline-block'}),
    html.Div([
        dcc.Dropdown(
            id='y-axis',
            options=[{'label': col, 'value': col} for col in df.columns],
            value=df.columns[1]
        )
    ],
    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
    html.Div([
        dcc.Dropdown(
            id='chart-type',
            options=[
                {'label': 'Gráfico de Barras', 'value': 'bar'},
                #{'label': 'Gráfico de Curve', 'value': 'Curve'},
                {'label': 'Gráfico de Dispersión', 'value': 'scatter'},
                {'label': 'Gráfico de Pastel', 'value': 'pie'}
            ],
            value='scatter'
        )
    ],
        style={'width': '48%', 'display': 'inline-block'}),
    dcc.Graph(id='graph')
])


# Callback para mostrar los valores únicos en texto y el diccionario correspondiente
@app.callback(
    [Output('attribute-table', 'children'),
     Output('graph', 'figure')],
    [Input('x-axis', 'value'),
     Input('y-axis', 'value'),
     Input('chart-type', 'value')])

def show_attribute_values(x_axis, y_axis, chart_type):
    attribute_values = pd.DataFrame(columns=['Atributo', 'Nombre', 'Valores'])

    for attribute, conversion in conversion_dict.items():
        title = conversion['title']
        values = conversion['values']
        
        if attribute in lista:
            formatted_values = 'Min: {}, Max: {}'.format(values['min'], values['max'])
        else:
            unique_values = df[attribute].unique()
            formatted_values = [str(values[val]) if val in values else str(val) for val in unique_values]
            formatted_values = ', '.join(formatted_values)
            
        attribute_values = attribute_values.append({'Atributo': attribute, 'Nombre': title, 'Valores': formatted_values}, ignore_index=True)

    if chart_type == 'scatter':
        
        # Evitamos que los valores categóricos esten el eje de las x, no se representan bien para está gráfica
        if y_axis in categoricos:
            fig = px.scatter(df, x=x_axis, y=y_axis,color=y_axis, marginal_x="box")
        elif x_axis in categoricos:
            fig = px.scatter(df, x=y_axis, y=x_axis, color=x_axis, marginal_x="box")
        else:
            fig = px.scatter(df, x=x_axis, y=y_axis, color=y_axis, marginal_x="box", marginal_y="box")
            

            
    elif chart_type == 'bar':
        
        # Evitamos que los valores categóricos esten el eje de las x, no se representan bien para está gráfica
        if x_axis == y_axis:
            fig = px.histogram(df, x=x_axis, color=x_axis, histfunc = "count")
        elif y_axis in categoricos:
            fig = px.histogram(df, x=x_axis, color=y_axis, barmode='group')
        elif x_axis in categoricos:
            fig = px.histogram(df, x=y_axis, color=x_axis, barmode='group')
        else:
            fig = px.bar(df, x=x_axis, y=y_axis)

    elif chart_type == 'pie':
        if y_axis in categoricos:
            fig = px.pie(df, names=df[y_axis], title=f"Distribución de {y_axis}")
        else:
            fig = px.pie(df, names=y_axis, title=f"Distribución de {y_axis}")
        
    fig.update_layout(title='Atributo {} & Atributo {}'.format(x_axis, y_axis)
                        )
    
    return generate_attribute_table(attribute_values), fig


# Función para generar la tabla de atributos
def generate_attribute_table(df):
    table = html.Table([
        html.Thead(html.Tr([
            html.Th('Atributo', style=attribute_name_style),
            html.Th('Nombre', style=attribute_name_style),
            html.Th('Valores', style=attribute_name_style)
        ])),
        html.Tbody([
            html.Tr([
                html.Td(attribute, style=attribute_title_style),
                html.Td(nombre, style=attribute_title_style),
                html.Td(values, style=attribute_values_style)
            ]) for attribute, nombre, values in zip(df['Atributo'], df['Nombre'], df['Valores'])
        ])
    ])

    return table


# Función para detener la ejecución de la aplicación Dash al capturar una señal de teclado
def stop_execution(signal, frame):
    os._exit(0)


# Capturar la señal SIGINT (Ctrl+C) para detener la ejecución
signal.signal(signal.SIGINT, stop_execution)


def EjecutarApp(empezar):
 # Ejecutar la aplicación Dash
    if __name__ == empezar:
        app.run_server(debug=True, use_reloader=False)
    #app.run_server(debug=False)

In [7]:
#Ejecutar la aplicación Dash
EjecutarApp( '__main__')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_20700\595542344.py:122: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use